In [261]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [262]:
train = pd.read_csv('train.csv')

In [263]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [264]:
train = train[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
test = pd.read_csv('test.csv')
id = test['PassengerId']
test = test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]

In [265]:
train.shape

(891, 8)

In [266]:
train.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [267]:
train.dropna(axis=0, subset=['Embarked'])
train = pd.get_dummies(train, columns=['Sex', 'Embarked'])
train = train.drop(columns='Sex_female')
train = train.dropna(axis=0, subset=['Age'])
train['Age'] = train['Age'].fillna(train['Age'].median())

test.dropna(axis=0, subset=['Embarked'])
test = pd.get_dummies(test, columns=['Sex', 'Embarked'])
test = test.drop(columns='Sex_female')
test['Age'] = test['Age'].fillna(test['Age'].median())
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())

In [268]:
train.shape

(714, 10)

In [269]:
x = train.iloc[:, 1:]
y = train.iloc[:, 0]

In [270]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.3, random_state=0, stratify=y)

In [271]:
features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_male', 'Embarked_C',
       'Embarked_Q', 'Embarked_S']

In [272]:
'''
import lightgbm as lgb
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_val, y_val)
params = {'objective': 'binary'}

model = lgb.train(params, lgb_train)
'''

"\nimport lightgbm as lgb\nlgb_train = lgb.Dataset(x_train, y_train)\nlgb_eval = lgb.Dataset(x_val, y_val)\nparams = {'objective': 'binary'}\n\nmodel = lgb.train(params, lgb_train)\n"

In [273]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100, random_state=0)
forest.fit(x_train, y_train)

print('Accuracy on training set: ', forest.score(x_train, y_train))

Accuracy on training set:  0.9939879759519038


In [274]:
prediciton = forest.predict(test)

In [275]:
prediciton = pd.DataFrame(prediciton)


In [276]:
output = pd.concat([id, prediciton], axis=1)

In [277]:
output = output.rename(columns={0:'Survived'})

In [278]:
output

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [279]:
output.to_csv('submission.csv', index=False)